In [3]:
from scipy.optimize import fsolve
import math

In [4]:
m_bch = 14.8 # кг - масса боевой части
m_dop = 1.5 # кг - суммарная масса других элементов, входящих в отсек БЧ
rho_osk = 9800 # кг/м^3 - плотность ГПЭ

a, b, c = 20e-3, 10e-3, 50e-3 # м - грани параллепипеда ГПЭ
N = 80

m_gpe = N*a*b*c*rho_osk
m_gpe_1 = a*b*c*rho_osk
m_vv = m_bch - m_gpe - m_dop # кг - масса ВВ

D = 8.3 # км/с - скорость детонации ВВ

In [40]:
print(round(m_gpe, 5), m_gpe_1, m_vv)

7.84 0.098 5.46


Для ГПЭ теоретическая формула для определения их начальной скорости воспользуемся формулой для плоских тел

In [41]:
def get_V_ob(D, beta):
    return 1.2*D*(math.sqrt(1+32/27*beta) - 1)/(math.sqrt(1+32/27*beta) + 1)

def get_F(S_m, a, b, c):
    return S_m / (a*b*c)**(2/3)

def get_A(rho, S_m, c_x, m_gpe_1):
    up = rho*S_m*c_x
    down = 2*m_gpe_1
    return up/down

def get_C(S_m, H_0, m_gpe_1):
    return S_m*H_0/m_gpe_1

def get_V_psp(C, A, b_pregr, k):
    pow = 2*A*b_pregr/k
    return math.sqrt(C/A*(math.exp(pow) - 1))

def get_A1(rho_v, c_x, F, rho_osk, m_gpe_1):
    up = rho_v * c_x * F
    down = 2 * rho_osk * m_gpe_1
    return up/down

In [42]:
class obj:
    def __init__(self, b_pregr, rho_p, H_0, k, c_x, rho_v):
        self.b_pregr = b_pregr
        self.rho_p = rho_p
        self.H_0 = H_0
        self.k = k
        self.c_x = c_x
        self.rho_v = rho_v
        self.S_m = None
        self.F = None

        self.m_bch = 14.8 # кг - масса боевой части
        self.m_dop = 1.5 # кг - суммарная масса других элементов, входящих в отсек БЧ
        self.rho_osk = 9800 # кг/м^3 - плотность ГПЭ

        self.a, self.b, self.c = 20e-3, 10e-3, 50e-3 # м - грани параллепипеда ГПЭ
        self.N = 80

        self.D = 8.3 # км/с - скорость детонации ВВ
        
class Bairactar_(obj):
    def __init__(self, b_pregr, rho_p, H_0, k, c_x, rho_v):
        super().__init__(b_pregr, rho_p, H_0, k, c_x, rho_v)
    
    def equations(self, p):
        x = p
        return self.V_ob*1000*math.exp(-1*self.A1*x) - self.V_psp
    
    def calculate(self):
        self.m_gpe = self.N * self.a * self.b * self.c * self.rho_osk
        self.m_gpe_1 = self.a * self.b * self.c * self.rho_osk
        self.m_vv = self.m_bch - self.m_gpe - self.m_dop # кг - масса ВВ
        self.beta = self.m_vv / self.m_gpe
        self.V_ob = get_V_ob(self.D, self.beta)
        self.S_m = (self.a*self.b+self.a*self.c)/2
        self.F = get_F(self.S_m, self.a, self.b, self.c)
        self.A = get_A(self.rho_p, self.S_m, self.c_x, self.m_gpe_1)
        self.C = get_C(self.S_m, self.H_0, self.m_gpe_1)
        self.V_psp = get_V_psp(self.C, self.A, self.b_pregr, self.k)
        self.F = 1.487
        self.A1 = get_A1(self.rho_v, self.c_x, self.F, self.rho_osk, self.m_gpe_1)
        self.x =  fsolve(self.equations, (1))
        self.I_ub = self.x[0]

    def get_info(self):
        print('m_gpe =', self.m_gpe)
        print('m_gpe_1 =', self.m_gpe_1)
        print('m_vv =', self.m_vv)
        print('beta =', self.beta)
        print('V_ob =', self.V_ob)
        print('S_m =', self.S_m)
        print('F =', self.F)
        print('A =', self.A)
        print('C =', self.C)
        print('V_psp =', self.V_psp)
        print('A1 =', self.A1)
        print('self.I_ub =', self.I_ub)

class FF_18A(obj):
    def __init__(self, b_pregr, rho_p1, H_01):
        super().__init__(b_pregr, rho_p1, H_01)

In [43]:
Bairactar = Bairactar_(b_pregr=3, rho_p=1000, H_0=100, k=1.25, c_x=1.1, rho_v=1.213)
Bairactar.calculate()
Bairactar.get_info()

m_gpe = 7.840000000000001
m_gpe_1 = 0.098
m_vv = 5.46
beta = 0.6964285714285714
V_ob = 1.4872709647507776
S_m = 0.0006000000000000001
F = 1.487
A = 3.3673469387755106
C = 0.6122448979591837
V_psp = 1379.1995300250142
A1 = 0.0010329571532694713
self.I_ub = 73.032645060709


In [32]:
def equations(x):
    return x + 12 - y

y = 10
x =  fsolve(equations, 0)
print(x)

[-2.]
